# YOLO を使った物体検出

## インポート

In [ ]:
import os
from pathlib import Path

import certifi
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageColor

import tensorflow as tf

## セットアップ

In [ ]:
%%bash

if [[ ! -d "tensorflow-yolov4-tflite" ]]; then
    git clone --depth 1 https://github.com/hunglc007/tensorflow-yolov4-tflite.git
fi

In [ ]:
%%bash

cd tensorflow-yolov4-tflite
pip install -q -r requirements.txt

## モデルのダウンロード

In [ ]:
# ダウンロードに失敗しないようにするためのおまじない
os.environ["SSL_CERT_FILE"] = certifi.where()

tf.keras.utils.get_file(
    fname="yolov4.cfg",
    origin="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.cfg",
    cache_subdir="models/yolov4",
)
tf.keras.utils.get_file(
    fname="yolov4.weights",
    origin="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights",
    cache_subdir="models/yolov4",
)

print("Downloaded the model to ~/.keras/models/yolov4")

## モデルを SavedModel に変換

In [ ]:
%%bash

cd tensorflow-yolov4-tflite
python3 save_model.py --weights ~/.keras/models/yolov4/yolov4.weights --output ~/.keras/models/yolov4/saved_model --input_size 416 --model yolov4

## モデルの構造を確認

In [ ]:
%%bash

saved_model_cli show --all --dir ~/.keras/models/yolov4/saved_model

## ラベルマップデータをロード

In [ ]:
with Path("~/.keras/models/yolov4/coco.names").expanduser().open() as f:
    category_index = {id: {"id": id, "name": name.strip()} for id, name in enumerate(f)}

## モデルをロード

In [ ]:
model_dir = Path("~/.keras/models/yolov4/saved_model").expanduser()
model = tf.saved_model.load(str(model_dir))

## 画像データをロード

In [ ]:
image = Image.open("dog.jpg")
image

## 画像データを正規化

In [ ]:
images = tf.image.convert_image_dtype(np.asarray(image), dtype=tf.float32)
images = tf.image.resize(images, (416, 416))
images = tf.expand_dims(images, axis=0)
images.shape, images.dtype

## 推論を実行

In [ ]:
outputs = model(images)

boxes = tf.expand_dims(outputs[:, :, :4], axis=2)
scores = outputs[:, :, 4:]

boxes.shape, scores.shape

## Non-Maximun Suppression

In [ ]:
boxes, scores, labels, num_detections = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=20,
    max_total_size=20,
    score_threshold=0.5,
    iou_threshold=0.5,
)

boxes = boxes[0]
scores = scores[0]
labels = labels[0]
num_detections = int(num_detections[0])

num_detections, boxes.shape, scores.shape, labels.shape

## バウンディングボックスを描画

In [ ]:
output_image = image.copy()

draw = ImageDraw.Draw(output_image)
color = ImageColor.getrgb("#E91E63")
font = ImageFont.truetype("Monaco", size=46)

xy_scale = np.array([output_image.height, output_image.width, output_image.height, output_image.width])

for i in range(num_detections):
    label = int(labels[i])
    score = scores[i]

    if score <= 0.5:
        continue

    name = category_index[label]["name"]
    y_min, x_min, y_max, x_max = boxes[i] * xy_scale

    print(f"{name},{label},{score:.6f},{x_min:.6f},{y_min:.6f},{x_max:.6f},{y_max:.6f}")

    draw.text((x_min, y_min), name, font=font, fill=color)
    draw.rectangle((x_min, y_min, x_max, y_max), outline=color, width=4)

output_image